We'll download the HMDB-51 dataset (2GB) and extract videos into frames of images.

You need one HDD (slow) and one SSD (fast) storages. We'll make symbolic links into the `data/` directory.  
We assume the path of each is `/storage/kiyoon` and `/faster/kiyoon` respectively, but rename it regarding your system.

Note that the paths to directories are defined in `dataset_configs/hmdb.py`.

In [117]:
# Environments for future use
from pyvideoai.config import PYVIDEOAI_DIR, DATA_DIR, DEFAULT_EXPERIMENT_ROOT
%env PYVIDEOAI_DIR=$PYVIDEOAI_DIR
%env DATA_DIR=$DATA_DIR
%env EXPERIMENT_ROOT=$DEFAULT_EXPERIMENT_ROOT

# !! CHANGE BELOW
%env HDD_PATH=/storage/kiyoon
%env SSD_PATH=/faster/kiyoon

env: PYVIDEOAI_DIR=/home/kiyoon/project/PyVideoAI
env: DATA_DIR=/home/kiyoon/project/PyVideoAI/data
env: HDD_PATH=/storage/kiyoon
env: SSD_PATH=/faster/kiyoon


In [79]:
# Directory preparation
# Make soft links in data/ directory

!mkdir -p "$HDD_PATH/datasets/hmdb51"
!mkdir -p "$SSD_PATH/datasets/hmdb51/frames_q5"
!mkdir "$DATA_DIR"
!ln -s "$HDD_PATH/datasets/hmdb51" "$DATA_DIR/"
!ln -s "$SSD_PATH/datasets/hmdb51/frames_q5" "$DATA_DIR/hmdb51/"

# Make experiments output directory as well
!mkdir -p "$HDD_PATH/PyVideoAI_experiments"
!ln -s "$HDD_PATH/PyVideoAI_experiments" "$EXPERIMENT_ROOT"

Now that we have all directories setup, we'll download and extract the RAR archived files.

In [78]:
# Download HMDB dataset in `data/hmdb51`
# It will also download the splits and extract the rar files
!bash "$PYVIDEOAI_DIR/submodules/video_datasets_api/hmdb_tools/download_hmdb.sh" "$DATA_DIR/hmdb51"

--2021-06-13 03:52:39--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2021-06-13 03:52:40--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘/home/kiyoon/project/PyVideoAI/data/hmdb51/hmdb51_org.rar’

/home/kiyoon/projec 100%[===================>]   1.98G   682KB/s    in 38m 46s 

2021-06-13 04:31:27 (892 KB/s) - ‘/home/kiyoon/project/PyVideoAI/data/hmdb51/hmdb51_org.rar’ saved [2124008126/2124008126]

Extracting video data


You should see \*.avi videos in `data/hmdb51/videos`,  
and annotations saved in `data/hmdb51/testTrainMulti_7030_splits`.

Next, we're extracting videos into frames of images to `data/hmdb51/frames_q5` directory.

In [104]:
# Extract videos into frames of images (quality=5)
# This will be saved in the fast SSD storage.
# You can SKIP this if you want to use video decoding dataloader.
!bash "$PYVIDEOAI_DIR/submodules/video_datasets_api/hmdb_tools/hmdb_extract_frames.sh" "$DATA_DIR/hmdb51/videos" "$DATA_DIR/hmdb51/frames_q5" 5

1 / 6766
/home/kiyoon/project/PyVideoAI/data/hmdb51/frames_q5/pushup/push_ups_pushup_f_nm_np1_ri_goo_1
Average processing time per segment: 0.08, ETA: 518
2 / 6766
/home/kiyoon/project/PyVideoAI/data/hmdb51/frames_q5/pushup/push-up_or_shut_up_(IXK)_pushup_f_cm_np1_le_med_2
Average processing time per segment: 0.06, ETA: 413
3 / 6766
/home/kiyoon/project/PyVideoAI/data/hmdb51/frames_q5/pushup/100_Push-ups_Initial_Test_hundredpushups_com_pushup_f_nm_np1_fr_goo_1
Average processing time per segment: 0.05, ETA: 370
4 / 6766
/home/kiyoon/project/PyVideoAI/data/hmdb51/frames_q5/pushup/YOUTUBE_PUSH_UP_CHALLENGE_pushup_f_cm_np1_fr_med_1
Average processing time per segment: 0.05, ETA: 351
5 / 6766
/home/kiyoon/project/PyVideoAI/data/hmdb51/frames_q5/pushup/Ninja_Warrior_Ultimate_Push_up_Training_pushup_f_nm_np1_le_goo_1
Average processing time per segment: 0.05, ETA: 341
6 / 6766
/home/kiyoon/project/PyVideoAI/data/hmdb51/frames_q5/pushup/Push_Up_Battle_pushup_u_nm_np2_le_bad_0
Average processi

We'll convert the official split labels into the format that our dataloader expects, and we provide code for this.  
If you want to know the details of the format, below explains what it is like:

**Format used in** `FramesSparsesampleDataloader` in `dataloader/frames_sparsesample_dataloader.py`    
**or** `FramesDensesampleDataloader` in `dataloader/frames_densesample_dataloader.py`  

```
num_classes if multilabel else 0
path/to/frames/dir_1/{:05d}.jpg video_id_1 label_1 start_frame_idx_1 end_frame_idx_1
path/to/frames/dir_2/{:05d}.jpg video_id_2 label_2 start_frame_idx_2 end_frame_idx_2
...
path/to/frames/dir_N/{:05d}.jpg video_id_N label_N start_frame_idx_N end_frame_idx_N
```

### 1. First line: one-hot encoding or not

#### TL;DR
The first line has to be 0 for single-label classification, and `num_classes` for multi-label classification task.
In HMDB-51, it has to be 0.

#### Details
If the value is greater than 0, the dataloader will one-hot encode the labels and return as array of size `num_classes` (which is necessary for multi-label).  
If the value is 0, the dataloader will return the labels as integers (which is for single-label).

### 2. Second line ~ end
#### Image path
This has to be Python-style formattable string, where the {:05d} part will be replaced by the frame index with 5-length zero paddings.  
Also, this path can be relative and you can provide `path_prefix` argument to the dataloader.

#### Video ID
Here, the video_id has to be unique per video, and will be used for multiple-clip evaluation (multicrop evaluation).
To elaborate, the accuracy calculator will average the prediction of the videos with the same video ID,  
and the dataloader will automatically sample each video into multiple clips in different spatial-temporal location,  
so all you need to do is to make sure that the video ID is unique.

#### Label
One label integer for single-label, and comma-separated labels for multi-label task.

#### Start/end frame index
Video will be sampled from `start_frame_idx` to `end_frame_idx` (including edge indices).  
Since the extraction code above count frames from zero, the `start_frame_idx` has to be always zero in this HMDB-51 example.  
Note that when you have long untrimmed video, you can define subclips changing this starting and ending frame index.

In [110]:
# Generate splits in the FramesSparsesampleDataloader format (using the official splits)
# in `data/hmdb51/splits_frames` directory
!bash "$PYVIDEOAI_DIR/tools/datasets/hmdb_splits_to_csv_frame_extracted.sh" "$DATA_DIR/hmdb51/"{testTrainMulti_7030_splits,frames_q5,splits_frames}

brush_hair
cartwheel
catch
chew
clap
climb
climb_stairs
dive
draw_sword
dribble
drink
eat
fall_floor
fencing
flic_flac
golf
handstand
hit
hug
jump
kick
kick_ball
kiss
laugh
pick
pour
pullup
punch
push
pushup
ride_bike
ride_horse
run
shake_hands
shoot_ball
shoot_bow
shoot_gun
sit
situp
smile
smoke
somersault
stand
swing_baseball
sword
sword_exercise
talk
throw
turn
walk
wave


Let's take a look at how the actual CSV files look like.  
Note that the first line is 0, and the Video IDs are unique.

In [114]:
# Print 10 lines
!head "$DATA_DIR/hmdb51/splits_frames/train1.csv"

0
brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0/{:05d}.jpg 0 0 0 407
brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1/{:05d}.jpg 1 0 0 393
brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2/{:05d}.jpg 2 0 0 321
brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ri_med_3/{:05d}.jpg 3 0 0 157
brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_0/{:05d}.jpg 4 0 0 135
brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_1/{:05d}.jpg 5 0 0 357
brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2/{:05d}.jpg 6 0 0 189
brush_hair/Blonde_being_brushed_brush_hair_f_nm_np2_ri_med_0/{:05d}.jpg 7 0 0 77
brush_hair/Blonde_being_brushed_brush_hair_u_cm_np2_ri_med_1/{:05d}.jpg 8 0 0 272


Now the data preparation is complete!  
In the next tutorial, you'll learn how to run an example code with an example configuration!